## Libraries

In [37]:
import praw
import pandas as pd
import datetime as dt
import json
from bs4 import BeautifulSoup
import re
import nltk

## Get Cred

In [2]:
# Load credentials from json file\n
with open("reddit_credentials.json", "r") as file:
    creds = json.load(file)

In [3]:
creds

{'client_id': 'Tt3sc9zHX1U4Pg',
 'client_secret': 'Tl_rWZZtVo0k46FFkM2i0BBCWQM',
 'user_agent': 'Scraping_data',
 'username': '311Sheetal',
 'password': 'Reddit'}

In [4]:
reddit = praw.Reddit(client_id = creds['client_id'],
                     client_secret = creds['client_secret'],
                     user_agent = creds['user_agent'],
                     username = creds['username'],
                     password = creds['password'])

## Load Comments of a Post

In [5]:
def fetch_comments(reddit_instance,postids):
    
    comments_dict = {
        "created": [],
        "comment_id": [],
        "author": [],
        "body": [],
        "parent_id":[],
        "submission_id":[],
        "score":[],
        "subreddit":[],
        "subreddit_id":[]
    }

    submission = reddit_instance.submission(postids)
    for postid in postids:
        submission = reddit_instance.submission(postid)
        submission.comments.replace_more(limit=None)
        for comment in submission.comments.list():
            comments_dict['created'].append(comment.created_utc)
            comments_dict['comment_id'].append(comment.id)
            comments_dict['author'].append(comment.author)
            comments_dict['body'].append(comment.body)
            comments_dict['parent_id'].append(comment.parent_id)
            comments_dict['submission_id'].append(postid)
            comments_dict['score'].append(comment.score)
            comments_dict['subreddit'].append(comment.subreddit)
            comments_dict['subreddit_id'].append(comment.subreddit_id)
            
    return pd.DataFrame(comments_dict)

In [6]:
comments_info = fetch_comments(reddit, ['9u948a'])

In [26]:
stopwords = nltk.corpus.stopwords.words('english')

In [28]:
# remove stopwords
comments_info['processed_body'] = comments_info['body'].apply(lambda x: " ".join([word for word in x.split() if word not in stopwords]))

In [42]:
# process html
comments_info['processed_body'] = comments_info['processed_body'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())

In [43]:
# check linked users
comments_info['linked_users'] = comments_info['processed_body'].apply(lambda x: re.findall('/u/[A-Za-z0-9_-]+',x))
# check linked subreddits
comments_info['linked_subreddits'] = comments_info['processed_body'].apply(lambda x: re.findall('r/[A-Za-z0-9_-]+',x))

In [45]:
# remove urls
comments_info['processed_body'] = comments_info['processed_body'].apply(lambda x: re.sub('https?://[A-Za-z0-9./]+','',x))
comments_info['processed_body'] = comments_info['processed_body'].apply(lambda x: re.sub('http?://[A-Za-z0-9./]+','',x))

In [46]:
comments_info.head()

,created,comment_id,author,body,parent_id,submission_id,score,subreddit,subreddit_id,processed_body,linked_users
0,1.541387e+09,e92pd71,TheLizardKing25,I’m republican and glad she lost but this is a...,t3_9u948a,9u948a,187,Republican,t5_2qndt,I’m republican glad lost terrible comparison,[]
1,1.541424e+09,e93ctfa,Russilito,This is a bit of a stretch...,t3_9u948a,9u948a,11,Republican,t5_2qndt,This bit stretch...,[]
2,1.541393e+09,e92uzr2,TakenStankForever,Are you trying to tell me that UCF *aren't* na...,t3_9u948a,9u948a,3,Republican,t5_2qndt,Are trying tell UCF *aren't* national champs?,[]
3,1.541391e+09,e92tnh4,Poopsmith89,I dont completely understand the scoring syste...,t3_9u948a,9u948a,10,Republican,t5_2qndt,I dont completely understand scoring system (e...,[]
4,1.541386e+09,e92o8rr,FreedomOfOpression,But Hillary got more votes. Or Am I misunderst...,t3_9u948a,9u948a,16,Republican,t5_2qndt,But Hillary got votes. Or Am I misunderstandin...,[]
